In [109]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
from dataclasses import dataclass


from pa_lib.file import (
    project_dir,
    load_bin,
    load_pickle,
    load_xlsx,
    store_bin,
)
from pa_lib.data import (
    as_dtype,
    dtFactor,
    lookup,
    desc_col,
    chi2_expected,
    clean_up_categoricals,
)
from pa_lib.util import (
    cap_words,
    collect,
    value,
    normalize_rows,
    as_percent,
    flat_list,
    list_items,
)
from pa_lib.html import HR, html, H1, H2, H3

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data sets

In [20]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_pickle("spr_data.pkl")
    time_codes = load_pickle("time_code_ratios.pkl")
    station_codes = load_pickle("station_code_ratios.pkl")
    global_codes = load_pickle("global_code_ratios.pkl")
    population_codes = load_pickle("population_ratios.pkl")
    ax_var_struct = load_bin("ax_var_struct.feather")

11:44:12 [INFO] Started loading binary file ...
11:44:12 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
11:44:12 [INFO] ... finished loading binary file in 0.31s (1.17s CPU)
11:44:12 [INFO] Started loading pickle file ...
11:44:12 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.pkl
11:44:12 [INFO] ... finished loading pickle file in 0.01s (0.0s CPU)
11:44:12 [INFO] Started loading pickle file ...
11:44:12 [INFO] Reading from file C:\Users\kpf\data\axinova\time_code_ratios.pkl
11:44:12 [INFO] ... finished loading pickle file in 0.01s (0.02s CPU)
11:44:12 [INFO] Started loading pickle file ...
11:44:12 [INFO] Reading from file C:\Users\kpf\data\axinova\station_code_ratios.pkl
11:44:12 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
11:44:12 [INFO] Started loading pickle file ...
11:44:12 [INFO] Reading from file C:\Users\kpf\data\axinova\global_code_ratios.pkl
11:44:12 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
11:44:12 [INFO] St

## Select Axinova data by any column(s)

In [17]:
def _check_selection(data, selection, allowed_columns):
    allowed_values = {}
    for column in allowed_columns:
        allowed_values[column] = data[column].cat.categories
    if set(selection.keys()) - set(allowed_columns) != set():
        raise NameError(f"Illegal column name in selection: {selection.keys()}")
    clean_selection = {}
    for column in allowed_columns:
        if column in selection:
            col_values = flat_list(selection[column])
            if set(col_values) - set(allowed_values[column]) != set():
                raise ValueError(
                    f"Illegal value(s) in parameter {column}: {col_values}"
                )
            clean_selection[column] = col_values
        else:
            clean_selection[column] = None
    return clean_selection


def select_data(all_data, **selection):
    """
    Filter ax_data by different columns. Supports sequences of allowed values.
    """
    select_columns = (
        "DayOfWeek Station Variable Month TimeSlot Hour "
        + "Time TimeSlot_cat StationSprache Code".split()
    )
    selection = _check_selection(all_data, selection, allowed_columns=select_columns)
    row_mask = pd.Series([True] * all_data.shape[0])
    for col in select_columns:
        if selection[col] is not None:
            row_mask &= all_data[col].isin(selection[col])
    return all_data.loc[row_mask].pipe(clean_up_categoricals).reset_index(drop=True)

# Look up code ratios from Axinova data

In [4]:
RatioTable = pd.DataFrame


@dataclass
class Ratios:
    actual: RatioTable
    expected: RatioTable
    sd: RatioTable = None

In [5]:
def ax_population_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = population_codes.query("Variable == @variable").pivot_table(
        values="Pop_Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_global_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = global_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_station_ratios(variable: str, percent: bool = False) -> Ratios:
    actual_ratios = station_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Station", columns="Code", fill_value=0
    )
    expected_ratios = ax_global_ratios(variable)
    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios), expected=as_percent(expected_ratios)
        )
    else:
        result = Ratios(actual=actual_ratios, expected=expected_ratios)
    return result

In [6]:
def ax_ratios(
    variable: str,
    stations: str,
    reference: str = "all_stations",
    time_scale: str = "Hour",
    percent: bool = False,
) -> Ratios:
    subset = ax_data.loc[
        ax_data.Station.isin(flat_list(stations)) & (ax_data.Variable == variable)
    ]
    full_index = [
        (weekday, time)
        for weekday in ax_data["DayOfWeek"].cat.categories
        for time in ax_data[time_scale].cat.categories
    ]
    actual_counts = subset.pivot_table(
        values="Value",
        index=["DayOfWeek", time_scale],
        columns="Code",
        fill_value=0,
        aggfunc="sum",
    )
    actual_counts_sd_ratios = (
        (np.sqrt(actual_counts) / actual_counts)
        .fillna(0)
        .reindex(full_index, fill_value=0)
    )
    actual_ratios = normalize_rows(actual_counts).reindex(full_index, fill_value=0)

    if reference == "all_stations":
        expected_ratios = (
            time_codes[time_scale]
            .query("Variable == @variable")
            .pivot_table(
                values="Ratio",
                index=["DayOfWeek", time_scale],
                columns="Code",
                fill_value=0,
            )
        )
    elif reference == "station":
        expected_counts = chi2_expected(actual_counts)
        expected_ratios = normalize_rows(expected_counts).reindex(
            full_index, fill_value=0
        )
    else:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios),
            expected=as_percent(expected_ratios),
            sd=as_percent(actual_counts_sd_ratios),
        )
    else:
        result = Ratios(
            actual=actual_ratios, expected=expected_ratios, sd=actual_counts_sd_ratios
        )
    return result

# Look up SPR+ data split by variable

In [178]:
def spr_split(
    stations,
    variable,
    weekdays=None,
    reference="station",
    time_scale="Hour",
    type="abs",
    incl_totals=True,
    decimals=1,
):
    if type not in ["abs", "rel"]:
        raise ValueError(
            f"Parameter 'type' must be one of ('abs', 'rel'), was '{type}'"
        )
    if reference not in ["station", "all_stations"]:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )
    if weekdays is None:
        weekdays = [
            "Monday",
            "Tuesday",
            "Wednesday",
            "Thursday",
            "Friday",
            "Saturday",
            "Sunday",
        ]

    spr_counts = (
        spr_data.loc[
            spr_data.Station.isin(flat_list(stations))
            & spr_data.DayOfWeek.isin(weekdays)
        ]
        .groupby(["DayOfWeek", time_scale], observed=True)[["Total"]]
        .agg("sum")
    )
    spr_count_sd_ratios = (np.sqrt(spr_counts) / spr_counts).fillna(0)

    ratios = ax_ratios(
        stations=stations,
        variable=variable,
        reference=reference,
        time_scale=time_scale,
        percent=False,
    )
    if type == "abs":
        code_ratios = ratios.actual
    elif type == "rel":
        code_ratios = ratios.actual - ratios.expected
    split_counts = code_ratios.mul(spr_counts.Total, axis="index").round(decimals)

    sd_ratios = np.sqrt(
        (ratios.sd ** 2).add((spr_count_sd_ratios ** 2).values, axis="index")
    )

    if incl_totals:
        split_counts.set_axis(
            split_counts.columns.to_list(), axis="columns", inplace=True
        )
        split_counts["Total"] = spr_counts.round(decimals)
        sd_ratios.set_axis(sd_ratios.columns.to_list(), axis="columns", inplace=True)
        sd_ratios["Total"] = spr_count_sd_ratios.values
    if decimals == 0:
        split_counts = split_counts.astype("int")

    return split_counts, sd_ratios

# Tests

## Station ratios

In [8]:
with value(ax_station_ratios(variable="g_220", percent=True)) as autobesitz:
    display(autobesitz.actual)
    display(autobesitz.expected)
    display(autobesitz.actual.sub(autobesitz.expected.values, axis="columns"))

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,11
Bern,44,28,28
Biel/Bienne,50,25,25
Brig,52,39,9
Chur,46,37,17
Fribourg,31,56,13
Genève Aéroport,42,49,10


Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,-2,1,1
Basel SBB,-1,-9,10
Bellinzona,7,9,-15
Bern,-1,-1,2
Biel/Bienne,5,-4,-1
Brig,7,10,-17
Chur,1,8,-9
Fribourg,-14,27,-13
Genève Aéroport,-3,20,-16


## Time ratios for one station

In [ ]:
with value(
    ax_ratios(stations="Lausanne", variable="g_220", time_scale="TimeSlot")
) as lausanne_auto:
    display(lausanne_auto.actual)
    display(lausanne_auto.expected)
    display(lausanne_auto.actual - lausanne_auto.expected)

## SPR+ data for one station split by ratios

In [ ]:
with value(("Bern", "g_220", "TimeSlot")) as (stat, var, scale):
    display(spr_split(stations=stat, variable=var, type="abs", time_scale=scale)[0])
    display(spr_split(stations=stat, variable=var, type="rel", time_scale=scale)[0])

# Case study: Find best stations / time slots for owner of multiple cars

## Population ratios

In [10]:
ax_population_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,44,40,16


## Ratios over all stations

In [11]:
ax_global_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Multi-car owners are **underrepresented** at digital station panels (29% vs. 40%)

## Ratios per station

In [12]:
ax_station_ratios(variable="g_220", percent=True).actual

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,11
Bern,44,28,28
Biel/Bienne,50,25,25
Brig,52,39,9
Chur,46,37,17
Fribourg,31,56,13
Genève Aéroport,42,49,10


Best stations are **Fribourg** (56%), **Zürich Flughafen** (50%), and **Genève Aéroport** (49%).

This compares to 29% over all stations, and 40% in the base population.

## Time slots at one station: Zürich Flughafen

In [13]:
prozent = ax_ratios(
    variable="g_220", stations="Zürich Flughafen", time_scale="Hour", percent=True
).actual
personen = spr_split(
    stations="Zürich HB", variable="g_220", time_scale="Hour", type="abs", decimals=0
)[0]
time_analysis = prozent.merge(
    personen, left_index=True, right_index=True, suffixes=(" [%]", " [Pers.]")
)

time_analysis.loc[prozent["2+ Autos"] > 40]

1 Auto [%]  2+ Autos [%]  Keines [%]  1 Auto [Pers.]  \
DayOfWeek Hour                                                         
Monday    00            48            52           0            1123   
          01             0           100           0             802   
          04            18            82           0               0   
          05            29            60          11            1506   
          07            11            79          10           34559   
          09            38            59           3           14221   
          10            39            61           0           12515   
          11            44            52           4           17402   
          12            28            63           9           25315   
          13            31            67           3           19727   
          14            10            86           5           21865   
          15            34            58           7           25268   
          16            47            50           3           32116   
          19            18            49          33           26497   
          23            22            65          13            5848   
Tuesday   00             0           100           0            1211   
          04             0           100           0             491   
          05             6            79          15             955   
          06            18            62          19           13497   
          07            14            83           3           36538   
          08            33            63           3           31175   
          09            22            62          16           15898   
          10            40            48          12           13284   
          12            36            46          18           24729   
          13            16            72          12           21422   
          14            32            62           6           20304   
          15            38            56           6           18418   
          16            33            61           6           33664   
          18            25            46          30           42908   
          19            27            57          16           24160   
Wednesday 00            49            51           0             880   
          03             0           100           0             321   
          04            42            41          17             603   
          06            25            54          21           12555   
          07            15            65          21           33923   
          08            27            48          25           27911   
          10             7            87           6           15304   
          11            33            50          16           21522   
          12            51            46           3           25585   
          13            29            59          12           18061   
          14            43            52           5           22045   
          15            41            57           3           20312   
          16            41            49          11           29403   
          17            30            45          25           48907   
          18            19            68          14           36349   
          19            25            52          22           23459   
          20            39            46          15           18086   
          21            24            55          21           12219   
          22            42            55           3            7961   
          23            32            42          27            4302   
Thursday  00             0           100           0             982   
          03             0           100           0               0   
          04            31            69           0             597   
          07            40            56           5    

In [14]:
time_analysis.sort_values("2+ Autos [Pers.]", ascending=False).head(20)

1 Auto [%]  2+ Autos [%]  Keines [%]  1 Auto [Pers.]  \
DayOfWeek Hour                                                         
Wednesday 17            30            45          25           48907   
Tuesday   17            54            24          22           50686   
Thursday  16            42            52           6           32431   
          17            43            54           2           49443   
Wednesday 16            41            49          11           29403   
Friday    17            54            36          10           46106   
Monday    17            46            40          15           54606   
Tuesday   16            33            61           6           33664   
Sunday    16            32            40          28           33044   
Monday    16            47            50           3           32116   
Friday    18            45            45          10           32941   
Monday    08            59            28          14           26408   
Sunday    17            41            38          21           31467   
Wednesday 18            19            68          14           36349   
          08            27            48          25           27911   
Tuesday   08            33            63           3           31175   
Thursday  18            17            64          19           38275   
          08            69            14          17           26354   
Tuesday   18            25            46          30           42908   
Sunday    15            27            36          36           29123   

                2+ Autos [Pers.]  Keines [Pers.]  Total  
DayOfWeek Hour                                           
Wednesday 17               27117           20611  96636  
Tuesday   17               26275           21909  98870  
Thursday  16               22583           14328  69342  
          17               22268           23538  95249  
Wednesday 16               21033           18917  69354  
Friday    17               20395           23423  89924  
Monday    17               20102           21380  96088  
Tuesday   16               20031           16344  70039  
Sunday    16               19763           22988  75796  
Monday    16               19505           18034  69656  
Friday    18               18725           23042  74708  
Monday    08               17822           13943  58173  
Sunday    17               17437           27605  76509  
Wednesday 18               17393           26159  79901  
          08               17215           11452  56578  
Tuesday   08               16811           10889  58874  
Thursday  18               16756           24741  79773  
          08               16730           13662  56745  
Tuesday   18               16607           21488  81002  
Sunday    15               15868           25693  70684

## Time slots with CI at one station: Zürich Flughafen

In [15]:
(counts, sd_ratios) = spr_split(
    stations="Zürich Flughafen",
    variable="g_220",
    time_scale="Hour",
    type="abs",
    decimals=1,
)

counts.merge(
    as_percent(sd_ratios),
    left_index=True,
    right_index=True,
    suffixes=(" [Pers]", " [sd%]"),
)

1 Auto [Pers]  2+ Autos [Pers]  Keines [Pers]  Total [Pers]  \
DayOfWeek Hour                                                                
Monday    00             11.3             12.3            0.0          23.6   
          01              0.0              0.0            0.0           0.0   
          02              0.0              0.0            0.0           0.0   
          03              0.0              0.0            0.0           0.0   
          04              0.0              0.0            0.0           0.0   
          05             17.1             35.5            6.8          59.5   
          06             85.7             72.7           75.0         233.4   
          07             56.8            414.8           55.1         526.6   
          08            289.5            137.2           67.0         493.8   
          09            124.5            191.7           10.3         326.4   
          10            114.7            179.9            0.0         294.5   
          11            145.6            174.1           14.4         334.1   
          12            122.4            274.6           39.2         436.3   
          13            114.6            247.7            9.9         372.2   
          14             34.9            306.8           16.7         358.3   
          15            138.1            234.5           28.7         401.2   
          16            265.9            279.7           15.2         560.8   
          17            349.1            302.9          114.1         766.1   
          18            197.5            187.5          245.3         630.2   
          19             73.6            202.8          133.8         410.2   
          20            135.8             76.9           69.8         282.5   
          21             68.8             52.4           74.3         195.5   
          22             14.2             37.2           81.7         133.1   
          23             14.8             44.9            8.8          68.5   
Tuesday   00              0.0             19.1            0.0          19.1   
          01              0.0              0.0            0.0           0.0   
          02              0.0              0.0            0.0           0.0   
          03              0.0              0.0            0.0           0.0   
          04              0.0              0.0            0.0           0.0   
          05              3.6             43.4            8.2          55.2   
          06             43.8            149.0           45.8         238.6   
          07             77.8            456.8           16.6         551.2   
          08            168.8            321.2           16.9         507.0   
          09             70.8            198.9           49.5         319.2   
          10            111.0            132.0           34.7         277.6   
          11            142.8            122.1           54.2         319.0   
          12            150.2            196.0           75.5         421.7   
          13             57.0            261.2           42.4         360.5   
          14            109.3            211.2           22.2         342.6   
          15            149.9            218.4           24.8         393.1   
          16            184.1            341.6           34.7         560.5   
          17            426.3            185.4          172.7         784.4   
          18            159.8            295.9          193.8         649.5   
          19            111.8            240.0           68.3         420.0   
          20            163.5             74.5           51.0         288.9   
          21             58.7             82.7           63.1         204.6   
          22             73.8             42.9           25.3         142.0   
          23             76.4              0.0            0.0          76.4   
Wednesday 00             10.8             11.1            0.0     

## Analysis for ZRH Airport (ax_data only)

In [16]:
with value(
    ax_ratios(
        variable="g_220",
        stations="Zürich Flughafen - Airside",
        reference="station",
        time_scale="TimeSlot",
    )
) as ratios:
    display(H3("Actual ratios"))
    display(ratios.actual)
    display(HR(), H3("Confidence intervals (sd in %)"))
    display(as_percent(ratios.sd))

Code                     1 Auto  2+ Autos    Keines
DayOfWeek TimeSlot                                 
Monday    Nacht        0.180987  0.552202  0.266811
          Morgen-Rush  0.273142  0.434854  0.292004
          Morgen       0.429608  0.413985  0.156407
          Mittag       0.160982  0.609423  0.229595
          Nachmittag   0.448211  0.480042  0.071747
          Abend-Rush   0.371419  0.478662  0.149919
          Abend        0.366501  0.421789  0.211709
Tuesday   Nacht        0.330247  0.669753  0.000000
          Morgen-Rush  0.505201  0.343728  0.151071
          Morgen       0.473495  0.470133  0.056372
          Mittag       0.281580  0.653515  0.064906
          Nachmittag   0.230271  0.685160  0.084569
          Abend-Rush   0.571696  0.278415  0.149889
          Abend        0.502403  0.389184  0.108413
Wednesday Nacht        0.318483  0.603296  0.078221
          Morgen-Rush  0.339760  0.506542  0.153698
          Morgen       0.441297  0.495463  0.063241
          Mittag       0.389913  0.542922  0.067165
          Nachmittag   0.287621  0.637168  0.075211
          Abend-Rush   0.380782  0.475836  0.143382
          Abend        0.426989  0.535527  0.037485
Thursday  Nacht        0.131547  0.638583  0.229871
          Morgen-Rush  0.474298  0.349632  0.176070
          Morgen       0.524723  0.362134  0.113143
          Mittag       0.498468  0.377932  0.123599
          Nachmittag   0.328184  0.509764  0.162052
          Abend-Rush   0.534097  0.373879  0.092024
          Abend        0.357598  0.477775  0.164627
Friday    Nacht        0.419110  0.536325  0.044564
          Morgen-Rush  0.382463  0.440346  0.177191
          Morgen       0.398114  0.587831  0.014055
          Mittag       0.422843  0.403243  0.173914
          Nachmittag   0.478173  0.428999  0.092828
          Abend-Rush   0.427678  0.308815  0.263506
          Abend        0.391722  0.364702  0.243576
Saturday  Nacht        0.345496  0.542165  0.112339
          Morgen-Rush  0.484003  0.381428  0.134569
          Morgen       0.355406  0.486557  0.158037
          Mittag       0.462163  0.426079  0.111758
          Nachmittag   0.456889  0.405917  0.137194
          Abend-Rush   0.464603  0.352769  0.182627
          Abend        0.531247  0.424358  0.044394
Sunday    Nacht        0.244906  0.621796  0.133298
          Morgen-Rush  0.348888  0.432377  0.218735
          Morgen       0.435467  0.329458  0.235076
          Mittag       0.504870  0.271625  0.223504
          Nachmittag   0.403811  0.412572  0.183618
          Abend-Rush   0.378084  0.404782  0.217134
          Abend        0.445524  0.384337  0.170139

Code                   1 Auto  2+ Autos  Keines
DayOfWeek TimeSlot                             
Monday    Nacht            27        16      22
          Morgen-Rush      15        12      15
          Morgen           17        18      29
          Mittag           26        13      22
          Nachmittag       13        12      32
          Abend-Rush       15        13      23
          Abend            14        13      18
Tuesday   Nacht            25        17       0
          Morgen-Rush      13        15      23
          Morgen           21        21      60
          Mittag           22        15      47
          Nachmittag       19        11      32
          Abend-Rush       15        22      30
          Abend            13        14      27
Wednesday Nacht            24        17      48
          Morgen-Rush      16        13      24
          Morgen           16        16      44
          Mittag           15        13      36
          Nachmittag       18        12      35
          Abend-Rush       15        13      25
          Abend            15        13      50
Thursday  Nacht            58        26      44
          Morgen-Rush      15        17      24
          Morgen           16        19      34
          Mittag           14        16      29
          Nachmittag       15        12      21
          Abend-Rush       12        14      29
          Abend            16        14      24
Friday    Nacht            16        14      50
          Morgen-Rush      12        11      18
          Morgen           19        15     100
          Mittag           17        17      26
          Nachmittag       12        13      27
          Abend-Rush       13        16      17
          Abend            13        14      17
Saturday  Nacht            24        19      42
          Morgen-Rush      11        13      21
          Morgen           15        13      22
          Mittag           12        13      25
          Nachmittag       12        12      21
          Abend-Rush       14        16      22
          Abend            13        15      47
Sunday    Nacht            28        17      37
          Morgen-Rush      16        14      20
          Morgen           15        17      20
          Mittag           10        14      15
          Nachmittag       10        10      15
          Abend-Rush       11        11      15
          Abend            11        12      17

# Target audience ratios

In [83]:
var_info = {}
for var, data in ax_var_struct.groupby("Variable"):
    var_info[var] = dict(
        Label=data["Variable_Label"].max(),
        Codes=data["Label"].to_list(),
        Order=list(range(len(data["Label_Nr"].to_list()))),
    )


def var_label(variable):
    return var_info[variable]["Label"]


def var_code_labels(variable):
    return var_info[variable]["Codes"]


def var_code_order(variable):
    return var_info[variable]["Order"]

In [176]:
def target_ratios(variable, code_index, station, weekdays=None, time_scale="Hour"):
    code_nr = flat_list(code_index)
    weekday_list = [
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
    ]
    if weekdays is not None:
        if set(flat_list(weekdays)) - set(weekday_list) != set():
            raise ValueError(f"Illegal weekday in {weekdays}")
        weekday_list = flat_list(weekdays)

    try:
        code_labels = list_items(var_code_labels(variable), code_nr)
    except IndexError:
        raise ValueError(
            f"Illegal code index(es) in {code_nr}, allowed are {var_code_order(variable)}"
        ) from None

    def percent(num):
        return round(num * 100, 1)

    glob_ratio = percent(
        ax_global_ratios(variable)[code_labels].sum(axis="columns").values[0]
    )
    pop_ratio = percent(
        ax_population_ratios(variable)[code_labels].sum(axis="columns").values[0]
    )
    station_counts, sd_ratios = spr_split(
        stations=station,
        weekdays=weekday_list,
        variable=variable,
        time_scale=time_scale,
        incl_totals=True,
    )
    target_counts = station_counts[code_labels].sum(axis="columns")
    target_ratios = (
        normalize_rows(station_counts).mul(2)[code_labels].sum(axis="columns")
    )
    target_data = (
        pd.DataFrame({"Persons": target_counts, "Ratio": target_ratios})
        .eval("CI = sqrt(Persons)")
        .eval("CI_prc = 100 * CI / Persons")
    )

    display(HR())
    print(
        f"{var_label(variable)}{code_labels}, bhf = {glob_ratio}%, pop = {pop_ratio}%"
    )
    display(H3(station))
    display(target_data.head(24))

In [179]:
code_spec = {"g_220": (0, 1), "md_ek": [5], "md_hhverm": [5, 6]}
stations = ["Biel/Bienne"]
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

for (var, code_idx) in code_spec.items():
    for station in flat_list(stations):
        target_ratios(
            variable=var, code_index=code_idx, station=station, weekdays="Tuesday", time_scale="Hour"
        )

ValueError: Unable to coerce to DataFrame, shape must be (168, 3): given (24, 1)